<h1>Tree of Thought in Prompt Engineering</h1>


Keep your API ket in env file



In [2]:
!pip install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


In [3]:
!pip install openai --upgrade

API key and Client Setup


In [5]:
import os
from openai import OpenAI
os.environ["OPENAI_API_KEY"]=key
client = OpenAI()


Creating Class

In [6]:
import openai
import time

class TreeOfThoughts:
    def __init__(self, prompt, max_depth=3, branch_factor=3, api_key=None):
        self.prompt = prompt
        self.max_depth = max_depth
        self.branch_factor = branch_factor
        self.tree = {"root": []}
        openai.api_key = api_key

    def generate_thought(self, parent_thought):
        prompt = f"Based on the thought '{parent_thought}', generate a new thought or idea:"
        response= client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            model="gpt-3.5-turbo",
        )

        return response.choices[0].message.content.strip()

    def evaluate_thought(self, thought):
        prompt = f"On a scale of 0 to 1, how promising is this thought for solving the problem '{self.prompt}'? Thought: '{thought}'\nJust respond with a number between 0 and 1."
        response= client.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            model="gpt-3.5-turbo",
        )
        try:
            score = float(response.choices[0].message.content.strip())
            return max(0, min(score, 1))  # Ensure score is between 0 and 1
        except ValueError:
            return 0.5  # Default score if parsing fails

    def expand_tree(self, node="root", depth=0):
        if depth >= self.max_depth:
            return

        if node not in self.tree:
            self.tree[node] = []

        for _ in range(self.branch_factor):
            new_thought = self.generate_thought(node)
            score = self.evaluate_thought(new_thought)
            self.tree[node].append((new_thought, score))

            if score > 0.7:  # Only expand promising thoughts
                self.expand_tree(new_thought, depth + 1)

            time.sleep(1)  # To avoid hitting API rate limits

    def best_path(self):
        path = ["root"]
        current = "root"
        while current in self.tree and self.tree[current]:
            best_thought = max(self.tree[current], key=lambda x: x[1])
            current = best_thought[0]
            path.append(current)
        return path

    def solve(self):
        self.expand_tree()
        return self.best_path()







In [ ]:
# Example usage
api_key = key
tot = TreeOfThoughts("How can we reduce plastic waste in oceans?", api_key=api_key)
solution_path = tot.solve()

In [7]:
from IPython.display import Markdown, display

# Create a markdown string
markdown_text = "### Best Solution Path:\n"
for step in solution_path:
    markdown_text += f"- {step}\n"



In [8]:

# Display the markdown
display(Markdown(markdown_text))

### Best Solution Path:
- root
- The thought of "root" can inspire the idea of exploring and understanding the fundamental origins and underlying principles of a concept or problem. By delving into the root causes and factors influencing a situation, one can gain deeper insights and develop more effective solutions.
- "Like a tree's roots anchoring it firmly in the ground, understanding the root of a problem can provide stability and clarity, allowing for growth and transformation to take place."
